In [4]:
from client import list_tools, call_tool
await list_tools()

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'})]

In [5]:
await call_tool("get_balance", {"name": "John"})

CallToolResult(meta=None, content=[TextContent(type='text', text='1000.0', annotations=None)], isError=False)

In [7]:
from agents import FunctionTool

openai_tools = []
import json

for tool in await list_tools():
    schema = tool.inputSchema
    schema["additionalProperties"] = False
    openai_tool = FunctionTool(
        name=tool.name,
        description=tool.description,
        params_json_schema=schema,
        on_invoke_tool=lambda ctx, args: call_tool(tool.name, json.loads(args)),
    )
    openai_tools.append(openai_tool)

openai_tools


[FunctionTool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', params_json_schema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function <lambda> at 0x13b830400>, strict_json_schema=True)]

In [8]:
from agents import Agent, Runner

In [9]:
account_manager = Agent(
    name="Account Manager",
    instructions="You are an account manager at a bank. You use the tools to tell a customer their balance.",
    tools=openai_tools)


In [10]:
result = await Runner.run(account_manager, "My name's Ed, how much do I have in my account?")
print(result.final_output)

Hi Ed, your account balance is $1000.00.
